In [85]:
import torch
from torch.utils.data import Dataset
import numpy as np
import segyio
import numpy as np
import os
import pandas as pd
import pywt
import matplotlib
import torch.nn.functional as F
from torch.utils.data import DataLoader
matplotlib.use('TkAgg')
import re

In [113]:
def read_log(well_path):
# 创建一个空列表来存储数据
    filtered_data = []
    # 打开文件
    with open(well_path, 'r') as file:
        # 逐行读取文件内容
        for line in file:
            # 去除行末的换行符并按空格分割行数据
            data = line.strip().split()
            # 如果数据行的第一列是数字（排除空行）
            if data and data[0].isdigit():
                # 如果是“结论”列，我们需要检查是否需要替换汉字为数字
                filtered_data.append(data[1:3] + [data[-1]])
    # 汉字数字对应的字典
    conversion_dict = {
        '水层': '6',
        '差气层': '1',
        '干层': '0',
        '特低渗气层': '2',
        '气水同层': '3',
        '含气水层': '5',
        '气层':'4'
    }
    # 替换汉字为数字
    for item in filtered_data:
        item[-1] = conversion_dict.get(item[-1], item[-1])
    return filtered_data
well_path = 'D:\桌面/2024王威本科毕设\HN\井/DF1-1-8.txt'
data = read_log(well_path)
data

[['1343.7', '1345.1', '0'],
 ['1348.0', '1353.8', '0'],
 ['1353.8', '1354.7', '4'],
 ['1356.4', '1358.2', '4'],
 ['1368.7', '1372.8', '4'],
 ['1372.8', '1376.1', '4'],
 ['1376.1', '1378.0', '4'],
 ['1378.0', '1379.6', '4'],
 ['1379.6', '1382.0', '4'],
 ['1382.0', '1389.5', '4'],
 ['1389.5', '1390.5', '0'],
 ['1390.5', '1392.4', '4'],
 ['1392.4', '1403.7', '0'],
 ['1403.7', '1406.0', '0'],
 ['1407.2', '1409.5', '0'],
 ['1409.5', '1411.4', '0'],
 ['1420.0', '1422.0', '0'],
 ['1423.6', '1429.0', '0'],
 ['1429.0', '1438.3', '0'],
 ['1438.3', '1443.4', '6'],
 ['1443.4', '1449.4', '6'],
 ['1449.4', '1456.0', '6'],
 ['1456.0', '1469.0', '6'],
 ['1469.0', '1477.6', '6'],
 ['1477.6', '1482.4', '0']]

In [130]:
names = os.listdir('D:/桌面/2024王威本科毕设/HN/井')                   
c = []
for name in names:
    path = 'D:\桌面/2024王威本科毕设\HN\井' + '/' + name
    c.append(path)
match = re.search(r'DF\d+-\d+-[A-Za-z\d]+', well_path)
well_name = match.group(0)
well_path = c[11]
data = read_log(well_path)
data


[['2586.9', '2598.3', '0'],
 ['2607.9', '2629.1', '0'],
 ['2721.4', '2735.9', '5'],
 ['2760.8', '2763.0', '5'],
 ['2763.0', '2765.8', '5'],
 ['2765.8', '2769.4', '5'],
 ['2769.4', '2773.3', '5'],
 ['2773.3', '2782.9', '5'],
 ['2782.9', '2799.3', '5'],
 ['2806.6', '2834.0', '5'],
 ['2834.0', '2846.3', '0'],
 ['2863.1', '2866.7', '5'],
 ['2891.8', '2906.2', '0'],
 ['2907.8', '2909.1', '0'],
 ['2909.1', '2913.4', '0'],
 ['2913.4', '2916.2', '4'],
 ['2963.9', '2965.2', '0'],
 ['2983.3', '2984.1', '0'],
 ['2986.3', '2987.1', '0'],
 ['2988.2', '2992.7', '0'],
 ['3003.8', '3005.9', '0'],
 ['3009.6', '3011.6', '0'],
 ['3017.5', '3028.1', '0'],
 ['3034.9', '3035.5', '0'],
 ['3075.4', '3076.1', '0'],
 ['3095.2', '3096.6', '0'],
 ['3122.6', '3128.9', '0'],
 ['3130.5', '3132.6', '0'],
 ['3133.4', '3135.2', '0'],
 ['3139.5', '3140.8', '0'],
 ['3140.8', '3143.2', '1'],
 ['3143.2', '3145.6', '0'],
 ['3145.6', '3155.2', '1'],
 ['3155.2', '3157.5', '0'],
 ['3171.9', '3173.7', '0'],
 ['3173.7', '3175.5'

In [94]:
def read_inf(inf_path):
    data_dict = {}
    # 打开文件
    with open(inf_path, 'r') as file:
        # 逐行读取文件内容
        for idx, line in enumerate(file):
            # 跳过第一行
            if idx == 0:
                continue
            # 去除行末的换行符并按空格分割行数据
            data = line.strip().split()
            # 如果数据行的长度大于等于3（排除空行或不完整行）
            if len(data) >= 3:
                # 提取前三列数据
                columns = data[:3]
                # 去除冒号
                columns[0] = columns[0].replace(':', '')
                # 将第一个名称作为键，后面两个数据作为值添加到字典中
                data_dict[columns[0]] = (columns[1], columns[2])
    return data_dict

dict = read_inf( 'D:/桌面/2024王威本科毕设/HN/井点坐标.txt')

inline = int((int(float(dict[well_name][0])) - 148525.5)/12.5) + 2880
xline = int(int(2068537.5 - float(dict[well_name][1]))/12.5) + 2870
dict


{'DF1-1-2': ('156058', '2060215'),
 'DF1-1-8': ('155824.7', '2053220.7'),
 'DF1-1-11': ('154188.5', '2061487.5'),
 'DF1-1-12': ('158366.7', '2065231.56'),
 'DF1-1-13': ('160151.6', '2059865.6'),
 'DF1-1-1C': ('157097.5', '2055952.7'),
 'DF17-2-1': ('161472.1', '2053838.4'),
 'DF11-2-1d': ('156358.6', '2064917.3'),
 'DF13-1-10': ('151205.83', '2061583.37'),
 'DF1-1-Z1': ('154699.6', '2058893'),
 'DF11-2-2': ('159025.1', '2058538.1'),
 'DF11-2-7': ('158613', '2065162.5')}

In [89]:
def read_segy_as_data(segyfile, inline_row, xline_row, inline, xline, xline_start):
    with segyio.open(segyfile,"r+",iline=inline_row, xline=xline_row) as sgydata:
        head_data = sgydata.iline[inline][xline-xline_start]         
    return head_data       #得到井数据相同位置的地震道数据
data = read_segy_as_data('D:/桌面/2024王威本科毕设/HN/CNOOC_DF11_3D_OBN_QPSDM_STK_RAW_D_FEB2023.segy',  189, 193, inline, xline, 2870)
data

array([-292.65698 , -270.11353 , -148.2142  , ...,   -7.569336,
         72.46167 ,  100.50116 ], dtype=float32)